In [23]:
import requests
import json
import os
import numpy as np
import pandas as pd

In [24]:
#Test url from atlas website to see if it works
url = "https://api.phila.gov/ais/v1/search/212578?gatekeeperKey=82fe014b6575b8c38b44235580bc8b11&include_units=true"
# Execute create deck url
response_data = requests.get(url).json()
response_data

{'search_type': 'pwd_parcel_id',
 'search_params': {'include_units': 'true'},
 'query': '212578',
 'normalized': '212578',
 'crs': {'type': 'link',
  'properties': {'type': 'proj4',
   'href': 'http://spatialreference.org/ref/epsg/4326/proj4/'}},
 'page': 1,
 'page_count': 1,
 'page_size': 8,
 'total_size': 8,
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'ais_feature_type': 'address',
   'match_type': 'exact_key',
   'properties': {'street_address': '800-12 SPRING GARDEN ST',
    'address_low': 800,
    'address_low_suffix': '',
    'address_low_frac': '',
    'address_high': 812,
    'street_predir': '',
    'street_name': 'SPRING GARDEN',
    'street_suffix': 'ST',
    'street_postdir': '',
    'unit_type': '',
    'unit_num': '',
    'street_full': 'SPRING GARDEN ST',
    'street_code': 73800,
    'seg_id': 440331,
    'zip_code': '19123',
    'zip_4': '2690',
    'usps_bldgfirm': 'PHILA TRAFFIC COURT',
    'usps_type': 'F',
    'election_block_id': '100401831'

In [25]:
#Test to see if I can get zoning id 
response_data['features'][0]['properties']['zoning']

'CMX-3'

In [26]:
#Check the csv file with the given addresses
list_df = pd.read_csv("OPA list.xlsx - Auction List.csv")
list_df

,OPA,Address,County,Base District Zoning
0,561401200,"1211 FAUNCE ST, 19111-2720",Philadelphia,NaN
1,344223300,"624 WYNNEWOOD RD, 19151-3843",Philadelphia,NaN
2,663388700,"3658 CANBY DR, 19154-2008",Philadelphia,NaN
3,651110700,"4523 MARPLE ST, 19136-3715",Philadelphia,NaN
4,532016200,"1312 WELLS ST, 19111-4922",Philadelphia,NaN
...,...,...,...,...
320,541049500,"1461 STEVENS ST, 19149-3227",Philadelphia,NaN
321,604191900,"5830 ADDISON ST, 19143-1204",Philadelphia,NaN
322,382232500,"3540 CONRAD ST, 19129-1639",Philadelphia,NaN
323,092121100,"751 W ALLENS LN, 19119-3311",Philadelphia,NaN


In [27]:
#Create code that iterates through addresses in df and inserts each into api address
def get_json(filepath):
    json_file = [] 
    df = pd.read_csv(filepath)
    for addresses in df['Address']:
        urlcall = f'https://api.phila.gov/ais/v1/search/{addresses}?gatekeeperKey=82fe014b6575b8c38b44235580bc8b11&include_units=true'
        urlcallJSON = requests.get(urlcall).json()
        json_file.append(urlcallJSON)
    return json_file

In [28]:
df_3 = pd.read_csv("philaforeclosures_SaleAuctions.xlsx - Auction List.csv")
df_3.head()

,Auction ID,OPA,Address,County
0,1036996,561401200,"1211 FAUNCE ST, 19111-2720",Philadelphia
1,1037120,344223300,"624 WYNNEWOOD RD, 19151-3843",Philadelphia
2,1037126,663388700,"3658 CANBY DR, 19154-2008",Philadelphia
3,1037338,651110700,"4523 MARPLE ST, 19136-3715",Philadelphia
4,1037344,532016200,"1312 WELLS ST, 19111-4922",Philadelphia


In [39]:
atlas = get_json("philaforeclosures_SaleAuctions.xlsx - Auction List.csv")

In [41]:
atlas

[{'search_type': 'address',
  'search_params': {'include_units': 'true'},
  'query': '1211 FAUNCE ST,  19111-2720',
  'normalized': '1211 FAUNCE ST',
  'crs': {'type': 'link',
   'properties': {'type': 'proj4',
    'href': 'http://spatialreference.org/ref/epsg/4326/proj4/'}},
  'page': 1,
  'page_count': 1,
  'page_size': 1,
  'total_size': 1,
  'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'ais_feature_type': 'address',
    'match_type': 'exact',
    'properties': {'street_address': '1211 FAUNCE ST',
     'address_low': 1211,
     'address_low_suffix': '',
     'address_low_frac': '',
     'address_high': None,
     'street_predir': '',
     'street_name': 'FAUNCE',
     'street_suffix': 'ST',
     'street_postdir': '',
     'unit_type': '',
     'unit_num': '',
     'street_full': 'FAUNCE ST',
     'street_code': 33660,
     'seg_id': 860285,
     'zip_code': '19111',
     'zip_4': '2720',
     'usps_bldgfirm': '',
     'usps_type': 'S',
     'election_block_id'

In [45]:
def get_zoning(filepath):
    zone = get_json(filepath)
    for index in range(len(zone)):
        try:
            zoning = zone[index]['features'][0]['properties']['zoning']
        except KeyError:
            print("na")
    return zoning

In [54]:
opa_3 = get_zoning("philaforeclosures_SaleAuctions.xlsx - Auction List.csv")

na
na
na
na
na
na
na
na


In [56]:
opa_3

'RSA-3'

In [52]:
def add_to_df(filepath):
    df = pd.read_csv(filepath)
    new_df = []
    df["Base District Zoning"] = get_zoning(filepath)
    new_df.append(df)
    return pd.DataFrame(new_df)


In [53]:
opa_list_3 = add_to_df("philaforeclosures_SaleAuctions.xlsx - Auction List.csv")

na
na
na
na
na
na
na
na


ValueError: Must pass 2-d input. shape=(1, 262, 5)

In [49]:
opa_list_3

In [20]:
opa_list_3.to_csv("opa_list_filtered_02.csv")

In [57]:
#Turn JSON file into a dataframe that returns the needed information
def create_df(filepath):
    zone = get_json(filepath)

    df = pd.read_csv(filepath)
    for index in range(len(zone)):
        try:
            opa = zone[index]['features'][0]['properties']['opa_account_num']
            address = zone[index]['query']
            district_zone = zone[index]['features'][0]['properties']['zoning']

            df["OPA"] = opa
            df['Address'] = address
            df['County'] = 'Philadelphia'
            df['Base District Zoning'] = district_zone
        
        except KeyError:
            print("na")
    
    return df

In [18]:
opa_list = create_df("OPA list.xlsx - Auction List.csv")
opa_list.head()

,OPA,Address,County,Base District Zoning
0,561401200,"1211 FAUNCE ST, 19111-2720",Philadelphia,RSA-3
1,344223300,"624 WYNNEWOOD RD, 19151-3843",Philadelphia,RSA-3
2,663388700,"3658 CANBY DR, 19154-2008",Philadelphia,RSA-4
3,651110700,"4523 MARPLE ST, 19136-3715",Philadelphia,RSA-5
4,532016200,"1312 WELLS ST, 19111-4922",Philadelphia,RSA-5


In [16]:
opa_list.to_csv('opa_list_filtered.csv')


In [58]:
opa_list_2 = create_df("philaforeclosures_SaleAuctions.xlsx - Auction List.csv")

na
na
na
na
na
na
na
na


In [59]:
opa_list_2.head()

,Auction ID,OPA,Address,County,Base District Zoning
0,1036996,234241000,"4626 PILLING ST, 19124-3202",Philadelphia,RSA-3
1,1037120,234241000,"4626 PILLING ST, 19124-3202",Philadelphia,RSA-3
2,1037126,234241000,"4626 PILLING ST, 19124-3202",Philadelphia,RSA-3
3,1037338,234241000,"4626 PILLING ST, 19124-3202",Philadelphia,RSA-3
4,1037344,234241000,"4626 PILLING ST, 19124-3202",Philadelphia,RSA-3


In [41]:
opa_list_2.to_csv("opa_list_filtered_02.csv")